<a href="https://colab.research.google.com/github/kirankamatmgm/steering-wheel-prediction/blob/master/Copy_of_Steering_wheel_angle_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''This project deals with steering wheel angle prediction in self driving cars.
'''
__author__ = "Prajwal, Meghana, Kiran Kamath, Dhanush B Raj"
__version__ = "1.0.1"
__email__ = "tkushal216@gmail.com"
__status__ = "Prototype"


In [0]:
!unzip /content/drive/My\ Drive/project_dataset/dataset.zip

In [0]:
!pwd

In [0]:
import cv2
import torch 
import torch.utils.data as data
import numpy as np
import csv

samples = []

# Step 1 : Reading the images from csv file
with open('/content/data/driving_log.csv') as csvfile:
  
  reader = csv.reader(csvfile)
  next(reader,None)
  for line in reader:
    samples.append(line)

# Step 2 : Dividing the dataset for Training and Testing

# 80 percent of total dataset for Training
# 20 percent of total dataset for Testing
train_len = int(0.8*len(samples))
test_len = len(samples) - train_len
train_samples, testing_samples = torch.utils.data.random_split(samples, lengths=[train_len, test_len])

In [0]:
import cv2
import torch
import torch.utils.data as data
import numpy as np

# Step 3a: Define the transformations process, augmentation and parameters for dataloader 
# What is augmentation ?
# Augmentation refers to the process of generating new training data from a smaller data set.
# Why cropping is done ?
# -> cropping, basically, helps the model to focus only on the road by taking away the sky and other distracting stuff in the image.
# Why flipping is done ?
# -> flip images at random and change the sign of the predicted angle to simulate driving in the opposite direction

def augment(imgName, angle):
  name = 'data/IMG/' + imgName.split('/')[-1]
  current_image = cv2.imread(name)
  #cropping
  # why those numbers ?
  # ->
  current_image = current_image[65:-25, :, :]
  #random fliping
  # why rand < 0.5 ?
  #-> 
  if np.random.rand() < 0.5:
    current_image = cv2.flip(current_image, 1)
    angle = angle * -1.0  
  return current_image, angle

In [0]:
class Dataset(data.Dataset):

    def __init__(self, samples, transform=None):

        self.samples = samples
        self.transform = transform

    def __getitem__(self, index):
      
        batch_samples = self.samples[index]
        
        steering_angle = float(batch_samples[3])
        
        center_img, steering_angle_center = augment(batch_samples[0], steering_angle)
        left_img, steering_angle_left = augment(batch_samples[1], steering_angle + 0.4)
        right_img, steering_angle_right = augment(batch_samples[2], steering_angle - 0.4)

        center_img = self.transform(center_img)
        left_img = self.transform(left_img)
        right_img = self.transform(right_img)

        return (center_img, steering_angle_center), (left_img, steering_angle_left), (right_img, steering_angle_right)
      
    def __len__(self):
        return len(self.samples)

In [0]:
# Step 3b : Creating generator using the dataloader to parallasize the process.
params = {'batch_size': 32,
          'shuffle': True,
          'num_workers': 4}

from torch.utils.data import DataLoader
import torchvision.transforms as transforms

transformations = transforms.Compose([transforms.Lambda(lambda x: (x / 255.0) - 0.5)])

training_set = Dataset(train_samples, transformations)
training_generator = data.DataLoader(training_set, **params)

testing_set = Dataset(testing_samples, transformations)
testing_generator = data.DataLoader(testing_set, **params)